In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml

In [1]:
from bs4 import BeautifulSoup
import datetime as d

import lxml
import requests as r
import json
import sys

import toJsonFile as tj

In [2]:
# Get all category links
def getCatLinks():
    main_url = 'http://edition.cnn.com/'
    res = r.get(main_url)
    res.encoding = 'unicode'
    soup = BeautifulSoup(res.text, 'lxml')
    tags = soup.select('.m-footer__bucket_group .m-footer__link')

    catList = []
    for tag in tags:
        if (tag['href'].split('.')[0] == '//money'):
            catList.append('http:%s' % (tag['href']))
        elif (tag['href'].split(':')[0] != 'http'):
            catList.append(main_url + '%s' % (tag['href'].lstrip('/')))
    return catList

In [3]:
#[DEBUG for getCatLinks()]
for link in getCatLinks():
    res = r.get(link)
    if res.status_code == 200:
        print('[ok!]' + link)
    else:
        print('[%s Fail!]' % (res.status_code) + link)

[ok!]http://edition.cnn.com/us
[ok!]http://edition.cnn.com/africa
[ok!]http://edition.cnn.com/americas
[ok!]http://edition.cnn.com/asia
[ok!]http://edition.cnn.com/china
[ok!]http://edition.cnn.com/europe
[ok!]http://edition.cnn.com/middle-east
[ok!]http://edition.cnn.com/opinions
[ok!]http://edition.cnn.com/specials/politics/president-donald-trump-45
[ok!]http://edition.cnn.com/specials/politics/congress-capitol-hill
[ok!]http://edition.cnn.com/specials/politics/us-security
[ok!]http://edition.cnn.com/specials/politics/supreme-court-nine
[ok!]http://edition.cnn.com/specials/politics/trumpmerica
[ok!]http://money.cnn.com/news/
[ok!]http://money.cnn.com/data/markets/
[ok!]http://money.cnn.com/technology/
[ok!]http://money.cnn.com/luxury/
[ok!]http://edition.cnn.com/entertainment/celebrities
[ok!]http://edition.cnn.com/entertainment/movies
[ok!]http://edition.cnn.com/entertainment/tv-shows
[ok!]http://edition.cnn.com/entertainment/culture
[ok!]http://money.cnn.com/media
[ok!]http://money

In [4]:
# Get all article links under the given category url
def getArticlesList(cat_url):

    articlesList = []

    res = r.get(cat_url)
    res.encoding = 'unicode'
    soup = BeautifulSoup(res.text, 'lxml')
    titles = soup.select('.cd__headline a')

    for article in titles:
        art_info = {}
        art_info['title'] = article.text
        art_info['source'] = 'CNN'

        if (article['href'].lstrip('/').split(':')[0] == 'https'):
            art_info['url'] = article['href'].lstrip('/')
        elif (article['href'].lstrip('/').split(':')[0] == 'http'):
            art_info['url'] = article['href'].lstrip('/')
        else:
            art_info['url'] = 'http://edition.cnn.com/' + \
                article['href'].lstrip('/')

        articlesList.append(art_info)

    return articlesList

In [5]:
#[DEBUG for getArticlesList()]
for catLink in getCatLinks()[0:1]:
    print(len(getArticlesList(catLink)))

    for art in getArticlesList(catLink):
        res = r.get(art['url'])
        if res.status_code == 200:
            print('[ok!]' + art['title'])
        else:
            print('[%s Fail!]' % (res.status_code) + art['url'])   

96
[ok!]Manhunt widens for Facebook murder suspect
[ok!]Facebook shooting victim was walking home after Easter meal
[ok!]Why Facebook still struggles to pull down murder videos quickly
[ok!]Cops: Facebook shooting suspect may have crossed state lines
[ok!]Opinion: Make it a crime to show killing on Facebook
[ok!]Prince documents unsealed: No meds in home were prescribed to him
[ok!]Atlanta seems to be running out of interstates
[ok!]US sends newest F-35 stealth fighters to Europe
[ok!]How America executes its prisoners
[ok!]The Obamas are living the life nowadays
[ok!]NFL QB prospect is also a rocket scientist
[ok!]Kenyans sweep Boston Marathon
[ok!]Homeless, terminally ill dogs find sanctuary here
[ok!]2 Detroit police officers shot, wounded in burglary call
[ok!]As advisers cross globe to hotspots, Trump stays put
[ok!]Does mental competence matter in Arkansas executions?
[ok!]Are Gen Xers, boomers suffering serious psychological distress?  
[ok!]April the giraffe finally gives birth

In [6]:
# Get one single article content
def getArticle(art_url):

    art_dict = {}

    res = r.get(art_url)
    res.encoding = 'unicode'
    soup = BeautifulSoup(res.text, 'lxml')

    try:
        art_date = soup.find(itemprop='datePublished')['content'].split('T')[0]
    except:
        art_date = d.date.today().strftime('%Y/%m/%d')

    art_title = soup.select('title')[0].text
    art_body = soup.select('#body-text .zn-body__paragraph')

    art_paras = {}
    for numPara in range(len(art_body)):
        art_paras[numPara + 1] = art_body[numPara].text.strip()

    art_dict['title'] = art_title
    art_dict['url'] = art_url
    art_dict['source'] = 'CNN'
    art_dict['date'] = art_date
    art_dict['content'] = art_paras

    return art_dict

In [7]:
# print('There are %s categories.'%(len(getCatLinks())))

cat_urls = getCatLinks()
empty = 0
success = 0

for cat_url in cat_urls[0:2]:
    articles = getArticlesList(cat_url)
    print('There are %s articles in %s.'%(len(articles),cat_url))
    
    for article in articles:
        try:
            art_dict = getArticle(article['url'])

            if len(art_dict['content']) == 0:
                empty += 1
            else:
                success += 1
        except:
            print(sys.exc_info())
print('E:%s, S:%s'%(empty,success))

There are 96 articles in http://edition.cnn.com/us.
There are 46 articles in http://edition.cnn.com/africa.
E:91, S:51
